In [17]:
import sys
import numpy as np 
import torch
# import spconv 

In [18]:
import spconv
%load_ext autoreload

%autoreload 2
# from spconv.conv import SparseConcat

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
sparse_shape = [10, 20,30]
coors1_np=np.array([0,0,0,0,
                    0,5,5,5,
                    1,2,2,3,
                    1,3,3,3,
            
                  
                  ]).reshape(-1,4)
coors2_np=np.array([0,0,0,0,
                    0,5,5,5,
                    1,4,5,6,
                    1,4,4,5,
                    1,4,4,4,
                    
                  ]).reshape(-1,4)
voxel_features1_np = np.array([1,1,
                             2,2,
                               3,3,
                               4,4,
                               ]).reshape(-1,2)
voxel_features2_np = np.array([11,11,
                             22,22,
                               33,33,
                               44,44,
                               55,55,
                               ]).reshape(-1,2)
batch_size=2
device='cpu'
dtype=torch.float32

In [41]:
# sparse_shape = torch.from_numpy(sparse_shape_np).to(device=device)
coors1=torch.from_numpy(coors1_np).to(device=device, dtype=dtype)
coors2=torch.from_numpy(coors2_np).to(device=device, dtype=dtype)

voxel_features1 = torch.from_numpy(voxel_features1_np).to(device=device, dtype=dtype).requires_grad_()
voxel_features2 = torch.from_numpy(voxel_features2_np).to(device=device, dtype=dtype).requires_grad_()

In [42]:

coors1 = coors1.int()
coors2 = coors2.int()
tensor1= spconv.SparseConvTensor(voxel_features1, coors1, sparse_shape,
                                      batch_size)
tensor2= spconv.SparseConvTensor(voxel_features2, coors2, sparse_shape,
                                      batch_size)

In [43]:
sp_cat = spconv.SparseConcat3d(indice_key='spcat0')
sp_conv = spconv.SubMConv3d( in_channels=2,
                 out_channels=2,
                 kernel_size=3,
                            indice_key='subm0')

In [44]:
t = sp_cat([tensor1, tensor2])
t_conv = sp_conv(tensor2)

In [45]:
t.features

tensor([[ 1.,  1., 11., 11.],
        [ 2.,  2., 22., 22.],
        [ 3.,  3.,  0.,  0.],
        [ 4.,  4.,  0.,  0.],
        [ 0.,  0., 33., 33.],
        [ 0.,  0., 44., 44.],
        [ 0.,  0., 55., 55.]], grad_fn=<SparseConcatFunctionBackward>)

In [55]:
tensor2.indice_dict['subm0'][2]
# tensor2.indice_dict['subm0'][3]

tensor([[[-1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1]],

        [[-1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1]],

        [[-1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1]],

        [[-1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1]],

        [[-1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1]],

        [[-1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1]],

        [[-1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1]],

        [[-1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1]],

        [[-1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1]],

        [[ 3, -1, -1, -1, -1],
         [ 2, -1, -1, -1, -1]],

        [[-1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1]],

        [[-1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1]],

        [[ 4, -1, -1, -1, -1],
         [ 3, -1, -1, -1, -1]],

        [[ 0,  1,  2,  3,  4],
         [ 0,  1,  2,  3,  4]],

        [[ 3, -1, -1, -1, -1],
         [ 4, -1, -1, -1, -1]],

        [[-1, -1, -1, -1, -1],
         

In [47]:
tensor1.features

tensor([[1., 1.],
        [2., 2.],
        [3., 3.],
        [4., 4.]], requires_grad=True)

In [48]:
tensor2.features

tensor([[11., 11.],
        [22., 22.],
        [33., 33.],
        [44., 44.],
        [55., 55.]], requires_grad=True)

In [49]:
tensor1.indice_dict['spcat0'][2]

tensor([[[ 0,  1,  2,  3,  4,  4,  4, -1, -1],
         [ 0,  1,  5,  5,  2,  3,  4, -1, -1]]], dtype=torch.int32)

In [50]:
tensor1.indice_dict['spcat0'][2].shape

torch.Size([1, 2, 9])

In [12]:
tensor1.indice_dict['spcat0'] # outids, indices1,indice_pairs, indice_pair_num,spatial_shape1

(tensor([[0, 0, 0, 0],
         [0, 5, 5, 5],
         [1, 2, 2, 3],
         [1, 3, 3, 3],
         [1, 4, 5, 6],
         [1, 4, 4, 4],
         [1, 4, 4, 4]], dtype=torch.int32), tensor([[0, 0, 0, 0],
         [0, 5, 5, 5],
         [1, 2, 2, 3],
         [1, 3, 3, 3]], dtype=torch.int32), tensor([[[ 0,  1,  2,  3,  4,  4,  4, -1, -1],
          [ 0,  1,  5,  5,  2,  3,  4, -1, -1]]], dtype=torch.int32), tensor([7], dtype=torch.int32), [10,
  20,
  30])

In [13]:
g_out = torch.rand_like(t.features)

t.features.backward(g_out)

In [14]:
g_out

tensor([[0.4548, 0.8329, 0.8482, 0.9255],
        [0.1255, 0.5623, 0.1243, 0.9724],
        [0.2756, 0.3747, 0.8976, 0.3978],
        [0.9606, 0.4339, 0.5857, 0.8963],
        [0.3222, 0.9786, 0.6724, 0.8704],
        [0.7855, 0.3705, 0.2618, 0.8621],
        [0.5532, 0.3118, 0.0257, 0.0718]])

In [15]:
tensor1.features.grad

tensor([[0.4548, 0.8329],
        [0.1255, 0.5623],
        [0.2756, 0.3747],
        [0.9606, 0.4339]])

In [16]:
tensor2.features.grad

tensor([[0.8482, 0.9255],
        [0.1243, 0.9724],
        [0.6724, 0.8704],
        [0.2618, 0.8621],
        [0.0257, 0.0718]])